In [1]:
# ---------------------------
# Cell 1: Imports & Load Data
# ---------------------------

import pandas as pd
import numpy as np
import re
import random
import matplotlib.pyplot as plt
import seaborn as sns
import joblib # Added import for joblib

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import FeatureUnion
from sklearn.svm import LinearSVC
from sklearn.metrics import f1_score, classification_report, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight

import warnings
warnings.filterwarnings("ignore")

# Load your dataset
df = pd.read_csv("/content/drive/MyDrive/SmartSpend/smart-spend/data/transactions.csv")
df.head()

,merchant,category
0,Starbucks Coffee,Food
1,McDonalds,Food
2,Subway Sandwich,Food
3,KFC Order,Food
4,Dominos Pizza,Food


In [2]:
# ---------------------------------
# Cell 2: Preprocessing Functions
# ---------------------------------

BRAND_NORMALIZATION = {
    "swgy": "swiggy",
    "swgy ordr": "swiggy order",
    "zomato": "zomato",
    "mcd": "mcdonalds",
    "brgr kng": "burger king",
    "k f c": "kfc",
    "domnos": "dominos",
    "dmart": "d mart",
    "big bazr": "big bazaar",
    "hpcl": "hindustan petroleum",
    "iocl": "indian oil",
    "bpcl": "bharat petroleum",
    "jio bp": "jiobp",
    "amzn": "amazon",
    "flipkart ord": "flipkart order",
    "ps store": "playstation store",
    "yt premium": "youtube premium",
    "1mg": "tata 1mg",
    "medplus": "medplus pharmacy",
}

HEALTHCARE_EXPANSION = {
    "dentl": "dental",
    "clnc": "clinic",
    "sessn": "session",
    "physio": "physiotherapy",
    "meds": "medicines",
    "dlry": "delivery",
    "opd": "outpatient department",
    "cbc": "complete blood count",
}

BILL_NORMALIZATION = {
    "elec": "electricity",
    "tneb": "electricity board",
    "ctw": "water board",
    "cc": "credit card",
    "subscrn": "subscription",
    "billfee": "bill fee",
}

def apply_dictionary(text, dictionary):
    for k, v in dictionary.items():
        if k in text:
            text = text.replace(k, v)
    return text

def clean_text(text):

    text = text.lower()

    # Apply normalizations
    text = apply_dictionary(text, BRAND_NORMALIZATION)
    text = apply_dictionary(text, HEALTHCARE_EXPANSION)
    text = apply_dictionary(text, BILL_NORMALIZATION)

    # Remove numbers + symbols
    text = re.sub(r'\d+', ' ', text)
    text = re.sub(r'[^a-zA-Z\s]', ' ', text)

    # Cleanup spaces
    text = re.sub(r'\s+', ' ', text).strip()

    return text

def preprocess_dataframe(df):
    df = df.copy()
    df["merchant"] = df["merchant"].astype(str).apply(clean_text)
    return df

df_clean = preprocess_dataframe(df)
df_clean.head()


,merchant,category
0,starbucks coffee,Food
1,mcdonaldsonalds,Food
2,subway sandwich,Food
3,kfc order,Food
4,dominos pizza,Food


In [3]:
# ---------------------------------------
# Cell 3: Synthetic Noise Augmentation
# ---------------------------------------

def add_noise(text):
    options = []

    # random deletion
    if len(text) > 4:
        idx = random.randint(0, len(text) - 1)
        options.append(text[:idx] + text[idx+1:])

    # character swap
    if len(text) > 5:
        i = random.randint(0, len(text)-2)
        s = list(text)
        s[i], s[i+1] = s[i+1], s[i]
        options.append("".join(s))

    # extra spaces
    options.append(text.replace(" ", "  "))

    # repeat a word
    w = text.split()
    if len(w) > 1:
        options.append(text + " " + random.choice(w))

    return random.choice(options)

aug_rows = []
for i, row in df_clean.iterrows():
    if random.random() < 0.30:  # augment 30%
        aug_rows.append([add_noise(row["merchant"]), row["category"]])

df_aug = pd.DataFrame(aug_rows, columns=["merchant", "category"])
df_final = pd.concat([df_clean, df_aug]).reset_index(drop=True)

print("Original:", df_clean.shape)
print("With Augmentation:", df_final.shape)


Original: (477, 2)
With Augmentation: (615, 2)


In [4]:
# ---------------------------------------
# Cell 4: Additional Synthetic Expansion
# ---------------------------------------

def simple_variations(text):
    return [
        text,
        text + " store",
        text + " center",
        text.replace(" ", ""),
        text + " india",
    ]

extra_rows = []
for i, row in df_clean.iterrows():
    for v in simple_variations(row["merchant"]):
        if random.random() < 0.20:  # generate 20%
            extra_rows.append([v, row["category"]])

df_syn = pd.DataFrame(extra_rows, columns=["merchant", "category"])
df_final = pd.concat([df_final, df_syn]).reset_index(drop=True)

print("After Synthetic Expansion:", df_final.shape)


After Synthetic Expansion: (1098, 2)


In [5]:
# --------------------------------------
# Cell 5: TF-IDF Feature Engineering
# --------------------------------------

tfidf_word = TfidfVectorizer(
    analyzer='word',
    ngram_range=(1,3),
    max_features=60000,
    sublinear_tf=True
)

tfidf_char = TfidfVectorizer(
    analyzer='char',
    ngram_range=(3,6),
    max_features=60000
)

tfidf = FeatureUnion([
    ('word', tfidf_word),
    ('char', tfidf_char)
])

X = tfidf.fit_transform(df_final["merchant"])
y = df_final["category"]

print("TF-IDF shape:", X.shape)


TF-IDF shape: (1098, 16332)


In [6]:
# -----------------------------------
# Cell 6: Train Linear SVM
# -----------------------------------

svm_model = LinearSVC(
    C=2.0,
    class_weight='balanced',
    random_state=42
)



In [7]:
# -----------------------------------
# Cell 7: Train LightGBM
# -----------------------------------

from lightgbm import LGBMClassifier

lgbm_model = LGBMClassifier(
    boosting_type='gbdt',
    objective='multiclass',
    num_class=y.nunique(),
    class_weight='balanced',
    learning_rate=0.1,
    n_estimators=400
)


In [8]:
# --------------------------------------------------------
# Cell 8: 5-Fold CV Comparison (SVM vs LightGBM)
# --------------------------------------------------------

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

svm_scores = []
lgbm_scores = []

for fold, (train_idx, test_idx) in enumerate(skf.split(X, y)):

    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    # Train SVM
    svm_model.fit(X_train, y_train)
    preds_svm = svm_model.predict(X_test)
    f1_svm = f1_score(y_test, preds_svm, average='macro')
    svm_scores.append(f1_svm)

    # Train LightGBM
    lgbm_model.fit(X_train, y_train)
    preds_lgbm = lgbm_model.predict(X_test)
    f1_lgbm = f1_score(y_test, preds_lgbm, average='macro')
    lgbm_scores.append(f1_lgbm)

    print(f"Fold {fold+1}: SVM={f1_svm:.4f} | LGBM={f1_lgbm:.4f}")

print("\nAvg SVM Macro F1:", np.mean(svm_scores))
print("Avg LightGBM Macro F1:", np.mean(lgbm_scores))


Streaming output truncated to the last 5000 lines.
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with posit

In [9]:
best_model = LinearSVC(
    C=2.0,
    class_weight='balanced',
    random_state=42
)

best_model.fit(X, y)


LinearSVC(C=2.0, class_weight='balanced', random_state=42)

In [10]:
import joblib
joblib.dump(best_model, "smartspend_final_svm.pkl")
joblib.dump(tfidf, "smartspend_tfidf.pkl")


['smartspend_tfidf.pkl']

#Test on clean data

In [11]:
test_df = pd.read_csv("/content/drive/MyDrive/SmartSpend/smart-spend/data/test_clean.csv")
test_df


,merchant,category
0,Cafe Barista Coffee,Food
1,Samosa Corner Shop,Food
2,LunchBox Kitchen,Food
3,Keventers Milkshake,Food
4,Burger Spot Express,Food
5,IKEA Furniture Store,Shopping
6,Boat Earbuds Online,Shopping
7,FirstCry Kidswear,Shopping
8,Westside Clothing,Shopping
9,Allen Solly Store,Shopping


In [12]:
test_clean = preprocess_dataframe(test_df.dropna(subset=['category']))

In [13]:
test_vec = tfidf.transform(test_clean["merchant"])
test_preds = best_model.predict(test_vec)


In [14]:
from sklearn.metrics import classification_report, f1_score

print(classification_report(test_clean["category"], test_preds))

               precision    recall  f1-score   support

        Bills       0.80      0.80      0.80         5
Entertainment       0.83      1.00      0.91         5
         Food       1.00      0.80      0.89         5
         Fuel       1.00      1.00      1.00         5
    Groceries       0.80      0.80      0.80         5
   Healthcare       1.00      1.00      1.00         5
     Shopping       0.80      0.80      0.80         5
       Travel       1.00      1.00      1.00         5

     accuracy                           0.90        40
    macro avg       0.90      0.90      0.90        40
 weighted avg       0.90      0.90      0.90        40



#NOISY TEST

In [15]:
# Load noisy test file
noisy_df = pd.read_csv("/content/drive/MyDrive/SmartSpend/smart-spend/data/test_noisy.csv")

# Drop rows with NaN in category column before preprocessing and evaluation
noisy_df_cleaned_for_eval = noisy_df.dropna(subset=['category'])

# Preprocess the cleaned DataFrame
noisy_clean = preprocess_dataframe(noisy_df_cleaned_for_eval)

# Vectorize
noisy_vec = tfidf.transform(noisy_clean["merchant"])

# Predict
noisy_preds = best_model.predict(noisy_vec)

# Evaluate using the cleaned category column
print(classification_report(noisy_clean["category"], noisy_preds))

               precision    recall  f1-score   support

        Bills       0.79      0.88      0.83        25
Entertainment       0.75      0.84      0.79        25
         Food       0.66      0.76      0.70        25
         Fuel       0.96      0.92      0.94        25
    Groceries       0.79      0.76      0.78        25
   Healthcare       0.88      0.88      0.88        25
     Shopping       0.95      0.72      0.82        25
       Travel       0.91      0.84      0.88        25

     accuracy                           0.82       200
    macro avg       0.84      0.83      0.83       200
 weighted avg       0.84      0.82      0.83       200



In [ ]:
import joblib

# Load your final trained model + TF-IDF
best_model = joblib.load("smartspend_final_svm.pkl")
tfidf = joblib.load("smartspend_tfidf.pkl")


In [16]:
import pandas as pd

# Load the noisy dataset
noisy_df = pd.read_csv("/content/drive/MyDrive/SmartSpend/smart-spend/data/test_noisy.csv")

print("Noisy Dataset Sample:")
noisy_df.head()


Noisy Dataset Sample:


,merchant,category
0,strbks cf#21,Food
1,strbks cfee 88,Food
2,cfe brsta ord22,Food
3,mcd brkfst,Food
4,mcd brkfst99,Food


In [17]:
# Preprocess using your noise-cleaning function
noisy_clean = preprocess_dataframe(noisy_df)


In [18]:
# Convert merchant text → TF-IDF vectors
noisy_vec = tfidf.transform(noisy_clean["merchant"])

# Predict category
noisy_preds = best_model.predict(noisy_vec)


In [19]:
# Load full noisy dataset
noisy_df = pd.read_csv("/content/drive/MyDrive/SmartSpend/smart-spend/data/test_noisy.csv")

# Shuffle so all categories appear
noisy_df = noisy_df.sample(frac=1, random_state=42).reset_index(drop=True)

# --- NEW: Normalize category names by stripping whitespace ---
noisy_df["category"] = noisy_df["category"].astype(str).str.strip()
# --- END NEW ---

# Preprocess and predict
noisy_clean = preprocess_dataframe(noisy_df)
noisy_vec = tfidf.transform(noisy_clean["merchant"])
noisy_preds = best_model.predict(noisy_vec)

# Evaluate correctly
from sklearn.metrics import classification_report, confusion_matrix

# Create a boolean mask to identify rows where 'category' is not NaN
not_nan_mask = noisy_df["category"].notna()

# Apply the mask to filter both the true labels and the predicted labels
y_true_filtered = noisy_df["category"][not_nan_mask]
y_pred_filtered = noisy_preds[not_nan_mask]

print("Classification Report on NOISY DATA:")
print(classification_report(y_true_filtered, y_pred_filtered))

print("\nConfusion Matrix:")
print(confusion_matrix(y_true_filtered, y_pred_filtered))

Classification Report on NOISY DATA:
               precision    recall  f1-score   support

        Bills       0.79      0.88      0.83        25
Entertainment       0.75      0.84      0.79        25
         Food       0.66      0.76      0.70        25
         Fuel       0.96      0.92      0.94        25
    Groceries       0.79      0.76      0.78        25
   Healthcare       0.88      0.88      0.88        25
     Shopping       0.95      0.72      0.82        25
       Travel       0.91      0.84      0.88        25

     accuracy                           0.82       200
    macro avg       0.84      0.83      0.83       200
 weighted avg       0.84      0.82      0.83       200


Confusion Matrix:
[[22  0  1  0  0  2  0  0]
 [ 1 21  2  0  0  0  0  1]
 [ 1  3 19  0  1  0  1  0]
 [ 0  0  0 23  0  1  0  1]
 [ 0  2  3  1 19  0  0  0]
 [ 1  1  1  0  0 22  0  0]
 [ 0  0  3  0  4  0 18  0]
 [ 3  1  0  0  0  0  0 21]]


In [20]:
# Inspect unique values and their counts in the filtered true labels
print("Unique categories in y_true_filtered:")
print(y_true_filtered.value_counts())

Unique categories in y_true_filtered:
category
Bills            25
Food             25
Shopping         25
Groceries        25
Healthcare       25
Fuel             25
Travel           25
Entertainment    25
Name: count, dtype: int64


In [21]:
results = pd.DataFrame({
    "merchant_raw": noisy_df["merchant"],
    "merchant_cleaned": noisy_clean["merchant"],
    "actual_category": noisy_df["category"],
    "predicted_category": noisy_preds
})

results.head(20)


,merchant_raw,merchant_cleaned,actual_category,predicted_category
0,axccrd fee,axcredit cardrd fee,Bills,Bills
1,brgr kng expr22,burger king expr,Food,Food
2,flpcrt shpng,flpcrt shpng,Shopping,Groceries
3,vgtbl cart9,vgtbl cart,Groceries,Groceries
4,mdpls med#2,mdpls med,Healthcare,Healthcare
5,jio bp fl#1,jiobp fl,Fuel,Fuel
6,airnd flt99,airnd flt,Travel,Travel
7,dryfrt shp2,dryfrt shp,Groceries,Groceries
8,fmrs mkt22,fmrs mkt,Groceries,Groceries
9,vjsl elec,vjsl electricity,Shopping,Shopping


In [ ]:
import os
import joblib

os.makedirs("/content/drive/MyDrive/SmartSpend/smart-spend/saved_model", exist_ok=True)

joblib.dump(best_model, "/content/drive/MyDrive/SmartSpend/smart-spend/saved_model/svm_model.pkl")
joblib.dump(tfidf, "/content/drive/MyDrive/SmartSpend/smart-spend/saved_model/tfidf_vectorizer.pkl")

print("Saved to:", os.listdir("saved_model"))


FileNotFoundError: [Errno 2] No such file or directory: 'saved_model'